In [1]:
from PIL import Image, ImageDraw, ImageFont
import os
import shutil
import numpy as np
from tqdm import tqdm
import cv2
import pandas as pd


target_dir = "./font_char_img"
ttf_file_list = os.listdir(target_dir)

refine_dir = "./refine_ttf_file"
if not os.path.isdir(refine_dir):
    os.mkdir(refine_dir)

In [2]:
f = open("./target_char.txt","r")
t_char = f.readline()
t_char_list = list(t_char.replace(" ",""))
# print(list(t_char.replace(" ","")))

In [3]:


## 1차 유효성 검사
def char_check(font,t_char_list):
    try:
        for ch in t_char_list:
            _ = font.getsize(ch)    
    except:
        return False
    return True


def img_pixel_serch(img):
    x= np.array(img)
    ret, thresh1 = cv2.threshold(x,127,255, cv2.THRESH_BINARY)
    if np.count_nonzero(thresh1)==32*32:
        return False
    return True
    
    
    
def pixel_check(font,t_char_list):
    new_char_str = ""
    not_char_str = ""
    for char in t_char_list:
        img = Image.new("L", (32,32), 255)
        draw = ImageDraw.Draw(img)
        draw.text((0, 0), char, font=font)
        flg = img_pixel_serch(img)
        if flg == True:
            new_char_str+=char
        else:
            not_char_str+=char
    return new_char_str,not_char_str



df = pd.DataFrame(columns=['name',"flg",'char','n_char'])



In [4]:
for i in range(0,len(ttf_file_list)):
    ttf_file_list[i] = ttf_file_list[i][:-3]+"ttf"
# print(ttf_file_list)

In [5]:
target_dir = "./ttf_file"


result_info = []

for file_name in tqdm(ttf_file_list):
    
    font = ImageFont.truetype(target_dir+"/"+file_name)
    ch_flg = char_check(font,t_char_list)
    # ch_flg = False
    font_char_str = ""
    if ch_flg == True:
        font_char_str,new_char_str = pixel_check(font,t_char_list)
    result_info.append({file_name:[ch_flg,font_char_str,new_char_str]})
    df = df.append({"name":file_name,"flg":ch_flg,"char":font_char_str,"n_char":new_char_str},ignore_index=True)
    

100%|██████████| 1393/1393 [10:17<00:00,  2.26it/s]


In [6]:
df.to_csv('./font_info.csv')

In [8]:
## 파일 옮기고, 텍스트 파일 생성
true_df = df[df['flg']==True]
true_df

for i in range(0,len(true_df)):
    shutil.copy(os.path.join(target_dir,true_df.iloc[i]['name']),os.path.join(refine_dir,true_df.iloc[i]['name']))
    f = open(os.path.join(refine_dir,true_df.iloc[i]['name'][:-3]+"txt"),"w")
    f.write(true_df.iloc[i]['char'])
    f.close()